In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ppscore as pps
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

%matplotlib inline

#### Carregando os dados

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
# Criando o df resposta
answer = pd.DataFrame()
answer['NU_INSCRICAO'] = df_test['NU_INSCRICAO']

#### Explorando os dados

In [4]:
df_train.shape

(13730, 167)

In [5]:
df_test.shape

(4576, 47)

A base de teste possui menos colunas que a base de treino. Nesse caso, vamos manter na base treino apenas as colunas em comum.

In [6]:
# Verificando se as colunas da base de teste estão na base de treino
print(set(df_test.columns).issubset(set(df_train.columns)))

True


In [7]:
colunas = list(df_test.columns) #Selecionando apenas as colunas que existem na base de teste
colunas.append('NU_NOTA_MT') #Adicionando a coluna 'target'

In [8]:
df_train = df_train[colunas] # Selecionando na base treino apenas as colunas escolhidas 
df_train.shape

(13730, 48)

In [9]:
#Dataframe auxiliar na análise
df_aux = pd.DataFrame({'Type': df_train.dtypes,
                      'Missing': df_train.isna().sum(),
                      'Size': df_train.shape[0],
                       'Unique': df_train.nunique()
                     })
df_aux['Missing_%']= df_aux.Missing/df_aux.Size * 100
df_aux.sort_values(by='Missing_%', ascending=False)

,Type,Missing,Size,Unique,Missing_%
TP_DEPENDENCIA_ADM_ESC,float64,9448,13730,4,68.812819
TP_ENSINO,float64,9448,13730,3,68.812819
Q027,object,7373,13730,13,53.699927
NU_NOTA_REDACAO,float64,3597,13730,53,26.198106
NU_NOTA_LC,float64,3597,13730,2774,26.198106
TP_STATUS_REDACAO,float64,3597,13730,9,26.198106
NU_NOTA_COMP1,float64,3597,13730,15,26.198106
NU_NOTA_COMP2,float64,3597,13730,13,26.198106
NU_NOTA_COMP3,float64,3597,13730,12,26.198106
NU_NOTA_COMP4,float64,3597,13730,14,26.198106


Alguns campos possuem um grande número dados faltantes. Vamos optar por excluir as colunas que possuem mais de 50% de dados faltantes.

In [10]:
#Apagando as colunas com muitos dados faltantes
df_train.drop(columns=['TP_DEPENDENCIA_ADM_ESC','TP_ENSINO','Q027'], inplace=True)
df_test.drop(columns=['TP_DEPENDENCIA_ADM_ESC','TP_ENSINO','Q027'], inplace=True)

In [11]:
# Dados faltantes restantes
df_train.isna().sum().sort_values(ascending = False)

NU_NOTA_MT           3597
NU_NOTA_REDACAO      3597
NU_NOTA_LC           3597
TP_STATUS_REDACAO    3597
NU_NOTA_COMP2        3597
NU_NOTA_COMP3        3597
NU_NOTA_COMP4        3597
NU_NOTA_COMP5        3597
NU_NOTA_COMP1        3597
NU_NOTA_CN           3389
NU_NOTA_CH           3389
TP_ANO_CONCLUIU         0
IN_DISLEXIA             0
IN_SURDEZ               0
IN_CEGUEIRA             0
IN_BAIXA_VISAO          0
IN_TREINEIRO            0
TP_ESCOLA               0
TP_NACIONALIDADE        0
TP_COR_RACA             0
CO_UF_RESIDENCIA        0
TP_ST_CONCLUSAO         0
SG_UF_RESIDENCIA        0
IN_SABATISTA            0
NU_IDADE                0
TP_SEXO                 0
IN_DISCALCULIA          0
CO_PROVA_CN             0
IN_GESTANTE             0
TP_LINGUA               0
Q026                    0
Q025                    0
Q024                    0
Q006                    0
Q002                    0
Q001                    0
CO_PROVA_MT             0
IN_IDOSO                0
CO_PROVA_LC 

In [12]:
# Preecher dados faltantes com zero 
df_train = df_train.fillna(0)
df_test = df_test.fillna(8)

In [13]:
# Verificando a correlação das variáveis com o target
df_train.corr()['NU_NOTA_MT'].drop('NU_NOTA_MT').sort_values()

NU_IDADE            -0.240874
TP_ANO_CONCLUIU     -0.191553
TP_LINGUA           -0.111428
TP_COR_RACA         -0.070239
TP_NACIONALIDADE    -0.033130
IN_GESTANTE         -0.028129
IN_IDOSO            -0.009061
IN_BAIXA_VISAO      -0.006546
IN_SURDEZ           -0.002617
IN_SABATISTA        -0.002457
CO_UF_RESIDENCIA     0.002613
IN_DISLEXIA          0.005259
IN_DISCALCULIA       0.008392
TP_ST_CONCLUSAO      0.023736
IN_TREINEIRO         0.078804
TP_ESCOLA            0.189084
TP_STATUS_REDACAO    0.564273
NU_NOTA_COMP5        0.722275
NU_NOTA_COMP3        0.854946
NU_NOTA_COMP2        0.862709
NU_NOTA_COMP4        0.866827
TP_PRESENCA_CN       0.874764
TP_PRESENCA_CH       0.874764
NU_NOTA_REDACAO      0.875929
NU_NOTA_COMP1        0.882746
NU_NOTA_CH           0.903608
NU_NOTA_CN           0.907039
TP_PRESENCA_LC       0.923641
NU_NOTA_LC           0.943954
IN_CEGUEIRA               NaN
Name: NU_NOTA_MT, dtype: float64

In [14]:
# Selecionando apenas as variáveis que possuem mais de 80% de correlação com o target
pearsoncorr = df_train.corr(method='pearson')
col = pearsoncorr.columns[(pearsoncorr.NU_NOTA_MT>0.80) | (pearsoncorr.NU_NOTA_MT<-0.80)]
col = col.tolist()

df_train = df_train[col]
df_train.head()

,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_REDACAO,NU_NOTA_MT
0,1,1,1,436.3,495.4,581.2,120.0,120.0,120.0,80.0,520.0,399.4
1,1,1,1,474.5,544.1,599.0,140.0,120.0,120.0,120.0,580.0,459.8
2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# colunas do df_test
df_test = df_test[df_train.columns[:-1]] 

In [17]:
y = df_train['NU_NOTA_MT']
x_train = df_train.drop(['NU_NOTA_MT'],axis=1)

model = LinearRegression().fit(x_train, y)

In [19]:
answer['NU_NOTA_MT'] = model.predict(df_test)

answer.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,431.922226
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,460.081929
2,b38a03232f43b11c9d0788abaf060f7366053b6d,574.879214
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,5.487475
4,715494628a50142ce8cb17191cfe6d0f3cae0934,541.847809


In [20]:
answer.to_csv('answer.csv',index=False)